# Phase 2B: Test Contrainte G₂ avec Vraies Données LMFDB

## Objectif

Tester si la contrainte **8×β₈ = 13×β₁₃ ≈ 36** émerge sur de VRAIES L-functions
téléchargées depuis **LMFDB** (L-functions and Modular Forms Database).

## Sources de Données

1. **LMFDB API** : https://www.lmfdb.org/api/
2. **Tables d'Odlyzko** : Zéros de ζ(s) haute précision
3. **mpmath** : Calcul direct si nécessaire

---

In [ ]:
# Installation
# !pip install requests numpy scipy matplotlib

In [ ]:
import numpy as np
import requests
import json
import time
import os
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# GPU si disponible
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("✅ GPU disponible")
except ImportError:
    GPU_AVAILABLE = False
    from scipy.sparse import csr_matrix as sp_csr
    from scipy.sparse.linalg import eigsh as sp_eigsh
    print("⚠️ GPU non disponible, utilisation CPU")

print(f"Backend: {'CuPy (GPU)' if GPU_AVAILABLE else 'SciPy (CPU)'}")

---

## 1. Client LMFDB Robuste

In [ ]:
class LMFDBClient:
    """
    Client robuste pour l'API LMFDB.
    
    Documentation API: https://www.lmfdb.org/api/
    """
    
    BASE_URL = "https://www.lmfdb.org/api"
    
    def __init__(self, cache_dir: str = "./lmfdb_cache"):
        self.cache_dir = cache_dir
        os.makedirs(cache_dir, exist_ok=True)
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'GIFT-Research/1.0 (Academic Research)'
        })
    
    def _get_cache_path(self, key: str) -> str:
        """Chemin du cache pour une clé donnée."""
        safe_key = key.replace('/', '_').replace('?', '_').replace('&', '_')
        return os.path.join(self.cache_dir, f"{safe_key}.json")
    
    def _load_cache(self, key: str) -> Optional[dict]:
        """Charge depuis le cache si disponible."""
        path = self._get_cache_path(key)
        if os.path.exists(path):
            with open(path, 'r') as f:
                return json.load(f)
        return None
    
    def _save_cache(self, key: str, data: dict):
        """Sauvegarde dans le cache."""
        path = self._get_cache_path(key)
        with open(path, 'w') as f:
            json.dump(data, f)
    
    def fetch(self, endpoint: str, params: dict = None, 
              use_cache: bool = True, retries: int = 3) -> Optional[dict]:
        """
        Fetch depuis l'API LMFDB avec retry et cache.
        """
        cache_key = f"{endpoint}_{json.dumps(params or {}, sort_keys=True)}"
        
        # Vérifier le cache
        if use_cache:
            cached = self._load_cache(cache_key)
            if cached:
                return cached
        
        # Construire l'URL
        url = f"{self.BASE_URL}/{endpoint}"
        if params:
            param_str = "&".join(f"{k}={v}" for k, v in params.items())
            url = f"{url}?{param_str}"
        
        # Fetch avec retry
        for attempt in range(retries):
            try:
                response = self.session.get(url, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    if use_cache:
                        self._save_cache(cache_key, data)
                    return data
                
                elif response.status_code == 429:  # Rate limit
                    wait = 2 ** attempt
                    print(f"   Rate limited, waiting {wait}s...")
                    time.sleep(wait)
                
                else:
                    print(f"   HTTP {response.status_code}")
                    return None
                    
            except requests.exceptions.Timeout:
                print(f"   Timeout (attempt {attempt + 1}/{retries})")
                time.sleep(1)
            except Exception as e:
                print(f"   Error: {e}")
                return None
        
        return None
    
    def get_dirichlet_character_zeros(self, modulus: int, char_index: int = 1,
                                       max_zeros: int = 200) -> Optional[np.ndarray]:
        """
        Récupère les zéros d'un caractère de Dirichlet L(s, χ).
        
        Args:
            modulus: Le conducteur (q)
            char_index: Index du caractère
            max_zeros: Nombre max de zéros
        """
        print(f"   Fetching Dirichlet L-function mod {modulus}, char {char_index}...")
        
        # LMFDB utilise un format de label spécifique
        # Format: "modulus.char_index" ou recherche par propriétés
        
        # Méthode 1: Recherche directe par label de caractère
        endpoint = "lfunc_lfunctions"
        params = {
            "degree": 1,
            "conductor": modulus,
            "_format": "json",
            "_fields": "label,positive_zeros,conductor,degree",
            "_limit": 5
        }
        
        data = self.fetch(endpoint, params)
        
        if data and 'data' in data and len(data['data']) > 0:
            for record in data['data']:
                zeros = record.get('positive_zeros', [])
                if zeros and len(zeros) > 10:
                    zeros = [float(z) for z in zeros[:max_zeros]]
                    print(f"   ✅ Got {len(zeros)} zeros")
                    return np.array(sorted(zeros))
        
        print(f"   ⚠️ No zeros found for mod {modulus}")
        return None
    
    def get_elliptic_curve_zeros(self, label: str, 
                                  max_zeros: int = 200) -> Optional[np.ndarray]:
        """
        Récupère les zéros de la L-function d'une courbe elliptique.
        
        Args:
            label: Label LMFDB (ex: "11.a1", "37.a1")
        """
        print(f"   Fetching elliptic curve {label}...")
        
        # Format du label pour L-function de courbe elliptique
        # Le label de la L-function est différent du label de la courbe
        
        endpoint = "lfunc_lfunctions"
        params = {
            "degree": 2,
            "_format": "json",
            "_fields": "label,positive_zeros,conductor,degree,origin",
            "origin": f"EllipticCurve/Q/{label.split('.')[0]}",
            "_limit": 10
        }
        
        data = self.fetch(endpoint, params)
        
        if data and 'data' in data:
            for record in data['data']:
                zeros = record.get('positive_zeros', [])
                if zeros and len(zeros) > 5:
                    zeros = [float(z) for z in zeros[:max_zeros]]
                    print(f"   ✅ Got {len(zeros)} zeros from {record.get('label', 'unknown')}")
                    return np.array(sorted(zeros))
        
        print(f"   ⚠️ No zeros found for curve {label}")
        return None
    
    def get_available_l_functions(self, degree: int = None, 
                                   conductor_max: int = 100,
                                   limit: int = 50) -> List[dict]:
        """
        Liste les L-functions disponibles avec zéros.
        """
        print(f"   Listing available L-functions (conductor ≤ {conductor_max})...")
        
        endpoint = "lfunc_lfunctions"
        params = {
            "_format": "json",
            "_fields": "label,conductor,degree,motivic_weight,positive_zeros",
            "conductor": f"1-{conductor_max}",
            "_limit": limit,
            "_sort": "conductor"
        }
        
        if degree is not None:
            params["degree"] = degree
        
        data = self.fetch(endpoint, params, use_cache=False)
        
        if data and 'data' in data:
            # Filtrer ceux qui ont des zéros
            with_zeros = [
                r for r in data['data'] 
                if r.get('positive_zeros') and len(r.get('positive_zeros', [])) > 10
            ]
            print(f"   ✅ Found {len(with_zeros)} L-functions with zeros")
            return with_zeros
        
        return []
    
    def get_zeros_by_label(self, label: str, max_zeros: int = 200) -> Optional[np.ndarray]:
        """
        Récupère les zéros par label LMFDB direct.
        """
        print(f"   Fetching zeros for {label}...")
        
        endpoint = "lfunc_lfunctions"
        params = {
            "label": label,
            "_format": "json",
            "_fields": "label,positive_zeros,conductor,degree"
        }
        
        data = self.fetch(endpoint, params)
        
        if data and 'data' in data and len(data['data']) > 0:
            record = data['data'][0]
            zeros = record.get('positive_zeros', [])
            if zeros:
                zeros = [float(z) for z in zeros[:max_zeros]]
                print(f"   ✅ Got {len(zeros)} zeros")
                return np.array(sorted(zeros))
        
        print(f"   ⚠️ No zeros for {label}")
        return None

In [ ]:
# Test du client
client = LMFDBClient()

print("\n📡 Test de connexion LMFDB...")
available = client.get_available_l_functions(conductor_max=50, limit=20)

if available:
    print(f"\n📋 Exemples de L-functions disponibles:")
    for lf in available[:5]:
        n_zeros = len(lf.get('positive_zeros', []))
        print(f"   {lf['label']}: conductor={lf['conductor']}, degree={lf['degree']}, {n_zeros} zeros")
else:
    print("\n⚠️ Impossible de se connecter à LMFDB - vérifiez votre connexion")

---

## 2. Téléchargement des Zéros de Riemann (Odlyzko)

In [ ]:
def download_odlyzko_zeros(n_zeros: int = 10000, cache_dir: str = "./lmfdb_cache") -> np.ndarray:
    """
    Télécharge les zéros de Riemann depuis les tables d'Odlyzko.
    """
    cache_path = os.path.join(cache_dir, f"riemann_zeros_{n_zeros}.npy")
    
    if os.path.exists(cache_path):
        print(f"✅ Chargé depuis cache: {n_zeros} zéros de ζ(s)")
        return np.load(cache_path)
    
    # URL des tables d'Odlyzko
    url = "http://www.dtc.umn.edu/~odlyzko/zeta_tables/zeros1"
    
    print(f"📥 Téléchargement des zéros de Riemann...")
    
    try:
        response = requests.get(url, timeout=60)
        if response.status_code == 200:
            lines = response.text.strip().split('\n')
            zeros = []
            for line in lines:
                line = line.strip()
                if line and not line.startswith('#'):
                    try:
                        zeros.append(float(line.split()[0]))
                    except:
                        continue
                if len(zeros) >= n_zeros:
                    break
            
            zeros = np.array(sorted(zeros)[:n_zeros])
            np.save(cache_path, zeros)
            print(f"✅ Téléchargé et sauvegardé: {len(zeros)} zéros")
            return zeros
    except Exception as e:
        print(f"❌ Erreur téléchargement: {e}")
    
    # Fallback: générer avec mpmath si disponible
    try:
        from mpmath import zetazero
        print(f"🔄 Calcul avec mpmath ({n_zeros} zéros)...")
        zeros = [float(zetazero(k).imag) for k in range(1, min(n_zeros, 500) + 1)]
        return np.array(zeros)
    except ImportError:
        print("❌ mpmath non disponible")
        return None

In [ ]:
# Télécharger les zéros de Riemann
riemann_zeros = download_odlyzko_zeros(10000)

if riemann_zeros is not None:
    print(f"\n📊 Zéros de Riemann:")
    print(f"   γ₁ = {riemann_zeros[0]:.6f}")
    print(f"   γ₁₀₀ = {riemann_zeros[99]:.6f}")
    print(f"   γ₁₀₀₀ = {riemann_zeros[999]:.6f}" if len(riemann_zeros) > 999 else "")

---

## 3. Testeur de Contrainte G₂

In [ ]:
class GIFTConstraintTester:
    """
    Teste la contrainte 8×β₈ = 13×β₁₃ ≈ 36 sur des L-functions.
    """
    
    GIFT_LAGS = [5, 8, 13, 27]
    TARGET_PRODUCT = 36  # h_G₂²
    
    def __init__(self, use_gpu: bool = True):
        self.use_gpu = use_gpu and GPU_AVAILABLE
    
    def build_H(self, N: int, betas: List[float], 
                alpha_T: float = 0.1, alpha_V: float = 1.0):
        """Construit l'opérateur H."""
        lags = self.GIFT_LAGS
        row, col, data = [], [], []
        
        # Cinétique
        for i in range(N):
            row.append(i); col.append(i); data.append(2.0 * alpha_T)
            if i > 0:
                row.append(i); col.append(i-1); data.append(-1.0 * alpha_T)
            if i < N-1:
                row.append(i); col.append(i+1); data.append(-1.0 * alpha_T)
        
        # Potentiel GIFT
        for lag, beta in zip(lags, betas):
            for i in range(lag, N):
                row.append(i); col.append(i-lag); data.append(beta * alpha_V)
                row.append(i-lag); col.append(i); data.append(beta * alpha_V)
        
        if self.use_gpu:
            return cp_csr((cp.array(data), (cp.array(row), cp.array(col))), shape=(N, N))
        else:
            return sp_csr((np.array(data), (np.array(row), np.array(col))), shape=(N, N))
    
    def compute_r2(self, H, zeros: np.ndarray, k: int) -> float:
        """Calcule R² entre spectre de H et zéros."""
        if self.use_gpu:
            eig, _ = cp_eigsh(H, k=k, which='SA')
            eig = cp.asnumpy(eig)
        else:
            eig, _ = sp_eigsh(H, k=k, which='SM')
        
        eig = np.sort(eig)
        gamma = zeros[:k]
        
        X = np.column_stack([gamma, np.ones(k)])
        params, _, _, _ = np.linalg.lstsq(X, eig, rcond=None)
        pred = X @ params
        
        ss_res = np.sum((eig - pred)**2)
        ss_tot = np.sum((eig - np.mean(eig))**2)
        
        return 1 - ss_res / ss_tot if ss_tot > 0 else 0
    
    def find_optimal_betas(self, zeros: np.ndarray, 
                           N_matrix: int = 300, k_eig: int = 40) -> Dict:
        """
        Trouve β₈, β₁₃ optimaux SANS imposer la contrainte.
        """
        if len(zeros) < k_eig + 30:
            k_eig = max(20, len(zeros) - 30)
        
        N_matrix = min(N_matrix, len(zeros))
        
        # Grid search
        beta8_range = np.linspace(2.0, 7.0, 12)
        beta13_range = np.linspace(1.0, 5.0, 12)
        
        best_r2 = -np.inf
        best_b8, best_b13 = 4.5, 36/13
        
        for b8 in beta8_range:
            for b13 in beta13_range:
                betas = [1.0, b8, b13, 0.037]
                H = self.build_H(N_matrix, betas)
                
                try:
                    r2 = self.compute_r2(H, zeros, k_eig)
                    if r2 > best_r2:
                        best_r2 = r2
                        best_b8, best_b13 = b8, b13
                except:
                    continue
        
        # Calculer avec contrainte GIFT (36)
        betas_gift = [1.0, 4.5, 36/13, 0.037]
        H_gift = self.build_H(N_matrix, betas_gift)
        r2_gift = self.compute_r2(H_gift, zeros, k_eig)
        
        prod8 = 8 * best_b8
        prod13 = 13 * best_b13
        
        return {
            'beta8_optimal': float(best_b8),
            'beta13_optimal': float(best_b13),
            'product_8_beta8': float(prod8),
            'product_13_beta13': float(prod13),
            'product_mean': float((prod8 + prod13) / 2),
            'product_ratio': float(prod8 / prod13) if prod13 != 0 else 0,
            'r2_optimal': float(best_r2),
            'r2_gift_36': float(r2_gift),
            'deviation_from_36_pct': float(abs((prod8 + prod13)/2 - 36) / 36 * 100),
            'n_zeros_used': len(zeros),
            'k_eigenvalues': k_eig
        }

In [ ]:
# Initialiser le testeur
tester = GIFTConstraintTester(use_gpu=GPU_AVAILABLE)

# Test sur Riemann
if riemann_zeros is not None and len(riemann_zeros) > 100:
    print("\n🧪 Test sur ζ(s) (Riemann)...")
    result_riemann = tester.find_optimal_betas(riemann_zeros, N_matrix=500, k_eig=50)
    
    print(f"\n📊 Résultats ζ(s):")
    print(f"   β₈ optimal = {result_riemann['beta8_optimal']:.3f}")
    print(f"   β₁₃ optimal = {result_riemann['beta13_optimal']:.3f}")
    print(f"   8×β₈ = {result_riemann['product_8_beta8']:.1f}")
    print(f"   13×β₁₃ = {result_riemann['product_13_beta13']:.1f}")
    print(f"   Moyenne = {result_riemann['product_mean']:.1f} (cible: 36)")
    print(f"   Déviation = {result_riemann['deviation_from_36_pct']:.1f}%")
    print(f"   R² optimal = {result_riemann['r2_optimal']:.4f}")
    print(f"   R² GIFT(36) = {result_riemann['r2_gift_36']:.4f}")

---

## 4. Test Massif sur L-Functions LMFDB

In [ ]:
def run_lmfdb_test(client: LMFDBClient, tester: GIFTConstraintTester,
                   max_conductor: int = 100, max_functions: int = 30) -> Dict:
    """
    Test massif sur les L-functions de LMFDB.
    """
    print("\n" + "="*70)
    print("TEST MASSIF SUR L-FUNCTIONS LMFDB")
    print("="*70)
    
    results = {}
    
    # 1. Zéros de Riemann (référence)
    if riemann_zeros is not None and len(riemann_zeros) > 100:
        print("\n📊 [1/N] Riemann ζ(s)")
        results['riemann_zeta'] = tester.find_optimal_betas(riemann_zeros)
        results['riemann_zeta']['label'] = 'riemann_zeta'
        results['riemann_zeta']['conductor'] = 1
        results['riemann_zeta']['degree'] = 1
        results['riemann_zeta']['type'] = 'riemann'
        print(f"   Produit moyen: {results['riemann_zeta']['product_mean']:.1f}")
    
    # 2. Récupérer les L-functions disponibles
    print(f"\n🔍 Recherche de L-functions (conductor ≤ {max_conductor})...")
    available = client.get_available_l_functions(
        conductor_max=max_conductor, 
        limit=max_functions * 2  # Demander plus car certains n'auront pas assez de zéros
    )
    
    if not available:
        print("⚠️ Aucune L-function trouvée sur LMFDB")
        return results
    
    # 3. Tester chaque L-function
    tested = 0
    for i, lf in enumerate(available):
        if tested >= max_functions:
            break
        
        label = lf.get('label', 'unknown')
        conductor = lf.get('conductor', 0)
        degree = lf.get('degree', 0)
        zeros = lf.get('positive_zeros', [])
        
        # Filtrer: besoin d'au moins 50 zéros
        if len(zeros) < 50:
            continue
        
        print(f"\n📊 [{tested+2}/N] {label} (cond={conductor}, deg={degree}, {len(zeros)} zeros)")
        
        try:
            zeros_array = np.array([float(z) for z in zeros])
            result = tester.find_optimal_betas(zeros_array, N_matrix=min(300, len(zeros)), k_eig=min(40, len(zeros)-10))
            
            result['label'] = label
            result['conductor'] = conductor
            result['degree'] = degree
            result['type'] = 'lmfdb'
            
            results[label] = result
            tested += 1
            
            print(f"   Produit moyen: {result['product_mean']:.1f} (dév. {result['deviation_from_36_pct']:.1f}%)")
            
        except Exception as e:
            print(f"   ❌ Erreur: {e}")
    
    print(f"\n✅ {len(results)} L-functions testées")
    return results

In [ ]:
# Lancer le test massif
print("🚀 Lancement du test LMFDB...")
print("   (Ceci peut prendre quelques minutes selon la connexion)\n")

lmfdb_results = run_lmfdb_test(client, tester, max_conductor=100, max_functions=25)

---

## 5. Analyse Statistique

In [ ]:
def analyze_lmfdb_results(results: Dict) -> Dict:
    """Analyse statistique des résultats LMFDB."""
    
    print("\n" + "="*70)
    print("ANALYSE STATISTIQUE DES RÉSULTATS")
    print("="*70)
    
    if not results:
        print("❌ Aucun résultat à analyser")
        return {}
    
    # Extraire les données
    products = [r['product_mean'] for r in results.values()]
    deviations = [r['deviation_from_36_pct'] for r in results.values()]
    r2_values = [r['r2_optimal'] for r in results.values()]
    conductors = [r.get('conductor', 0) for r in results.values()]
    
    # Statistiques globales
    print(f"\n📊 Statistiques globales ({len(results)} L-functions):")
    print(f"   Produit moyen global: {np.mean(products):.2f} ± {np.std(products):.2f}")
    print(f"   Déviation de 36: {np.mean(deviations):.1f}% ± {np.std(deviations):.1f}%")
    print(f"   R² moyen: {np.mean(r2_values):.4f}")
    
    # Comptage proche de 36
    close_10 = sum(1 for d in deviations if d < 10)
    close_20 = sum(1 for d in deviations if d < 20)
    close_30 = sum(1 for d in deviations if d < 30)
    
    print(f"\n📊 Proximité à 36:")
    print(f"   < 10% déviation: {close_10}/{len(results)} ({100*close_10/len(results):.0f}%)")
    print(f"   < 20% déviation: {close_20}/{len(results)} ({100*close_20/len(results):.0f}%)")
    print(f"   < 30% déviation: {close_30}/{len(results)} ({100*close_30/len(results):.0f}%)")
    
    # Tableau détaillé
    print("\n" + "="*70)
    print("TABLEAU DÉTAILLÉ")
    print("="*70)
    print(f"{'Label':<25} {'Cond.':<8} {'Deg.':<5} {'Prod.Moy':<10} {'Dév.%':<8} {'R²':<8}")
    print("-" * 70)
    
    for label, r in sorted(results.items(), key=lambda x: x[1].get('deviation_from_36_pct', 999)):
        star = "★" if r['deviation_from_36_pct'] < 15 else ""
        print(f"{label:<25} {r.get('conductor', '?'):<8} {r.get('degree', '?'):<5} "
              f"{r['product_mean']:<10.1f} {r['deviation_from_36_pct']:<8.1f} {r['r2_optimal']:<8.4f} {star}")
    
    # Test de normalité autour de 36
    print("\n" + "="*70)
    print("TEST STATISTIQUE")
    print("="*70)
    
    # t-test: moyenne = 36 ?
    from scipy.stats import ttest_1samp
    t_stat, p_value = ttest_1samp(products, 36)
    
    print(f"\n📐 Test t (H₀: moyenne = 36):")
    print(f"   t = {t_stat:.3f}, p-value = {p_value:.4f}")
    
    if p_value > 0.05:
        print(f"   ✅ On ne peut pas rejeter H₀ - moyenne compatible avec 36")
    else:
        print(f"   ⚠️ Moyenne significativement différente de 36")
    
    return {
        'n_functions': len(results),
        'mean_product': float(np.mean(products)),
        'std_product': float(np.std(products)),
        'mean_deviation': float(np.mean(deviations)),
        'close_to_36_10pct': close_10,
        'close_to_36_20pct': close_20,
        't_statistic': float(t_stat),
        'p_value': float(p_value)
    }

In [ ]:
# Analyser
analysis = analyze_lmfdb_results(lmfdb_results)

---

## 6. Visualisation

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    def plot_lmfdb_results(results: Dict):
        """Visualise les résultats."""
        if not results:
            print("Pas de données à visualiser")
            return
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        products = [r['product_mean'] for r in results.values()]
        deviations = [r['deviation_from_36_pct'] for r in results.values()]
        conductors = [r.get('conductor', 1) for r in results.values()]
        r2_values = [r['r2_optimal'] for r in results.values()]
        
        # 1. Histogramme des produits
        ax1 = axes[0, 0]
        ax1.hist(products, bins=15, edgecolor='black', alpha=0.7)
        ax1.axvline(x=36, color='red', linestyle='--', linewidth=2, label='Cible = 36')
        ax1.axvline(x=np.mean(products), color='green', linestyle='-', linewidth=2, 
                    label=f'Moyenne = {np.mean(products):.1f}')
        ax1.set_xlabel('Produit moyen (8×β₈ + 13×β₁₃)/2')
        ax1.set_ylabel('Fréquence')
        ax1.set_title('Distribution des Produits Optimaux')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Produit vs Conducteur
        ax2 = axes[0, 1]
        ax2.scatter(conductors, products, alpha=0.7, s=60)
        ax2.axhline(y=36, color='red', linestyle='--', label='Cible = 36')
        ax2.set_xlabel('Conducteur')
        ax2.set_ylabel('Produit moyen')
        ax2.set_title('Produit vs Conducteur')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # 3. Distribution des déviations
        ax3 = axes[1, 0]
        ax3.hist(deviations, bins=15, edgecolor='black', alpha=0.7, color='orange')
        ax3.axvline(x=np.mean(deviations), color='red', linestyle='-', linewidth=2,
                    label=f'Moyenne = {np.mean(deviations):.1f}%')
        ax3.set_xlabel('Déviation de 36 (%)')
        ax3.set_ylabel('Fréquence')
        ax3.set_title('Distribution des Déviations')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 4. R² vs Déviation
        ax4 = axes[1, 1]
        ax4.scatter(deviations, r2_values, alpha=0.7, s=60)
        ax4.set_xlabel('Déviation de 36 (%)')
        ax4.set_ylabel('R² optimal')
        ax4.set_title('Performance vs Proximité à 36')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('lmfdb_analysis.png', dpi=150)
        plt.show()
        
        print("📊 Visualisation sauvegardée: lmfdb_analysis.png")
    
    plot_lmfdb_results(lmfdb_results)
    
except ImportError:
    print("matplotlib non disponible")

---

## 7. Conclusion et Export

In [ ]:
def final_conclusion(results: Dict, analysis: Dict):
    """Conclusion finale."""
    
    print("\n" + "="*70)
    print("CONCLUSION PHASE 2B - DONNÉES LMFDB RÉELLES")
    print("="*70)
    
    if not results or not analysis:
        print("❌ Données insuffisantes pour conclure")
        return
    
    mean_prod = analysis.get('mean_product', 0)
    p_value = analysis.get('p_value', 1)
    close_20 = analysis.get('close_to_36_20pct', 0)
    n_total = analysis.get('n_functions', 1)
    
    print(f"\n📊 Résumé:")
    print(f"   {n_total} L-functions testées (données LMFDB réelles)")
    print(f"   Produit moyen: {mean_prod:.1f} (cible: 36)")
    print(f"   p-value (test t): {p_value:.4f}")
    print(f"   Proches de 36 (<20%): {close_20}/{n_total}")
    
    # Verdict
    if p_value > 0.05 and abs(mean_prod - 36) < 5:
        print(f"\n🎯 VERDICT: La contrainte ~36 semble UNIVERSELLE")
        print(f"   → Moyenne compatible avec 36 (p > 0.05)")
        print(f"   → Fort support pour l'interprétation G₂")
        verdict = "UNIVERSAL"
    elif close_20 / n_total > 0.5:
        print(f"\n⚠️ VERDICT: Tendance vers 36 mais avec dispersion")
        print(f"   → Plus de 50% des L-functions proches de 36")
        print(f"   → Structure présente mais pas exacte")
        verdict = "PARTIAL"
    else:
        print(f"\n❓ VERDICT: Résultats non concluants")
        print(f"   → Dispersion trop importante")
        print(f"   → Besoin de plus de données ou révision du modèle")
        verdict = "INCONCLUSIVE"
    
    # Export
    summary = {
        'verdict': verdict,
        'n_l_functions': n_total,
        'mean_product': float(mean_prod),
        'p_value': float(p_value),
        'analysis': analysis,
        'detailed_results': {k: {kk: vv for kk, vv in v.items() if kk != 'positive_zeros'} 
                            for k, v in results.items()}
    }
    
    with open('lmfdb_phase2b_results.json', 'w') as f:
        json.dump(summary, f, indent=2, default=float)
    
    print(f"\n💾 Résultats sauvegardés: lmfdb_phase2b_results.json")
    
    return summary

final_summary = final_conclusion(lmfdb_results, analysis)

In [ ]:
print("\n" + "="*70)
print("NOTEBOOK TERMINÉ")
print("="*70)
print("\nCe notebook a testé la contrainte G₂ (8×β₈ = 13×β₁₃ ≈ 36)")
print("sur de VRAIES L-functions téléchargées depuis LMFDB.")
print("\nFichiers générés:")
print("  - lmfdb_phase2b_results.json")
print("  - lmfdb_analysis.png")
print("  - Cache dans ./lmfdb_cache/")